In [808]:
import pandas as pd
import numpy as np
import json
import pprint
import pymongo
from pymongo import MongoClient

### Loading and cleaning the CO2 data

In [809]:
# Reading in the CO2 totals and CO2 per capita data
co_df = pd.read_csv('data/CO2_flat.csv')
pc_df = pd.read_csv('data/percapita_flat.csv')


In [810]:
# Adding identifier columns to the dataframes so that we can merge them later
co_df['Type'] = 'Total CO2'
pc_df['Type'] = 'PerCapita CO2'

In [811]:
# Drop the per capita column from the co_df as it will be maintained in the pc_df
co_df = co_df.drop(columns = ['Per Capita'])

In [812]:
# Look at nulls
co_df.isnull().sum()

# Where is the country code missing?
co_df[co_df['ISO 3166-1 alpha-3'].isnull()]

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Type
19312,French Equatorial Africa,NaN,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19313,French Equatorial Africa,NaN,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19314,French Equatorial Africa,NaN,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19315,French Equatorial Africa,NaN,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19316,French Equatorial Africa,NaN,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
...,...,...,...,...,...,...,...,...,...,...,...
47867,Ryukyu Islands,NaN,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
47868,Ryukyu Islands,NaN,2018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
47869,Ryukyu Islands,NaN,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
47870,Ryukyu Islands,NaN,2020,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2


In [813]:
# Look at nulls in the other dataframe - both CO2 dataframes have the same country codes and data missing
pc_df.isnull().sum()

Country                   0
ISO 3166-1 alpha-3     1632
Year                      0
Total                 44132
Coal                  45966
Oil                   46065
Gas                   46092
Cement                47656
Flaring               46160
Other                 61484
Type                      0
dtype: int64

In [814]:
# Drop the rows with null country codes
co_df = co_df.dropna(subset=['ISO 3166-1 alpha-3'])
pc_df = pc_df.dropna(subset=['ISO 3166-1 alpha-3'])

In [815]:
# Concatenate the data frames
all_co_df = pd.concat([co_df, pc_df], ignore_index=True)

# Melt the data frame, id_vars is what columns to keep, value_vars is what columns to melt
all_co_melted = pd.melt(all_co_df, id_vars=['Country', 'ISO 3166-1 alpha-3', 'Year', 'Type'],
                    value_vars=['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other'],
                    var_name='Category', value_name='Emission')

In [816]:
# Renaming columns for clarity
all_co_melted = all_co_melted.rename(columns = {'ISO 3166-1 alpha-3':'Country Code', 'Emission':'Value'})

all_co_melted.head()

,Country,Country Code,Year,Type,Category,Value
0,Afghanistan,AFG,1750,Total CO2,Total,0.0
1,Afghanistan,AFG,1751,Total CO2,Total,0.0
2,Afghanistan,AFG,1752,Total CO2,Total,0.0
3,Afghanistan,AFG,1753,Total CO2,Total,0.0
4,Afghanistan,AFG,1754,Total CO2,Total,0.0


In [817]:
# Shortening the time frame from 1999-2019
all_co_melted_short = all_co_melted[all_co_melted['Year'] >= 1999]

In [818]:
# Creating a copy of the slice of the dataframe
all_co_melted_short = all_co_melted_short.copy()

# Converting the data types to float
all_co_melted_short[['Value']] = all_co_melted_short[['Value']].astype(float)


In [819]:
# Drop the rows with World and International Transport values
all_co_ready = all_co_melted_short[all_co_melted['Country'] != 'World']
all_co_ready = all_co_melted_short[all_co_melted['Country'] != 'International transport']


/opt/anaconda3/envs/pythondata/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/opt/anaconda3/envs/pythondata/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [820]:
all_co_ready.head()


,Country,Country Code,Year,Type,Category,Value
249,Afghanistan,AFG,1999,Total CO2,Total,1.091640
250,Afghanistan,AFG,2000,Total CO2,Total,1.047128
251,Afghanistan,AFG,2001,Total CO2,Total,1.069098
252,Afghanistan,AFG,2002,Total CO2,Total,1.340995
253,Afghanistan,AFG,2003,Total CO2,Total,1.559602


### Bringing in the GDP and Population data
Bopth the GDP and Population data are from the same source and in the same format just pulled as different times as adding population to the dataset came later.

In [821]:
# Read in the GDP and pop data
gdp_df= pd.read_csv('data/gdp_data.csv')
pop_df = pd.read_csv('data/population.csv')

# Concatenate the data frames
all_gdp_df = pd.concat([gdp_df, pop_df], ignore_index=True)

In [822]:
pop_df.dtypes

Country Name     object
Country Code     object
Series Name      object
Series Code      object
2019 [YR2019]    object
                  ...  
1964 [YR1964]    object
1963 [YR1963]    object
1962 [YR1962]    object
1961 [YR1961]    object
1960 [YR1960]    object
Length: 64, dtype: object

In [823]:
# Setting the population values to integers with no decimals
for col in pop_df.columns[4:]:
    if np.issubdtype(pop_df[col].dtype, np.number):
        pop_df[col] = pop_df[col].round(0).astype(int)

In [824]:
# Cleaning up the column names
new_columns = []
for column in all_gdp_df.columns:
    new_columns.append(column.split('[')[0].strip())

all_gdp_df.columns = new_columns

# Renaming the series to match Type from the CO2 data
all_gdp_df = all_gdp_df.rename(columns = {'Series Name':'Category', 'Country Name':'Country'})

# Adding the Type column for merging later with the CO2 data, add it in column 2
all_gdp_df.insert(2, 'Type', 'GDP')

# Dropping the series code column
all_gdp_df_formatted = all_gdp_df.drop(columns = ['Series Code'])

all_gdp_df_formatted.head()

,Country,Country Code,Type,Category,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,AFG,GDP,GDP (current US$),537777811.111111,548888895.555556,546666677.777778,751111191.111111,800000044.444444,1006666637.77778,...,15633856787.0425,18190410820.6235,20203572959.5023,20564485419.1684,20550582746.8448,19998156214.3988,18019558181.9602,18896352021.94,18418848299.5825,18904490262.913
1,Afghanistan,AFG,GDP,GDP per capita (current US$),62.3693745050559,62.4437034626929,60.9503638210144,82.0217375781519,85.5110734102311,105.243195716467,...,554.595200222354,621.912310861592,663.141052810937,651.987861948108,628.146803888496,592.476537451681,520.252064031151,530.149830802984,502.056770622973,500.522664145294
2,Africa Eastern and Southern,AFE,GDP,GDP (current US$),21291524631.3606,21809435284.9773,23708060554.431,28211280561.2905,26119938954.6996,29683481336.0513,...,860361207235.72,964213016259.692,972002199045.518,982677082467.187,1003403000510.52,923143900034.922,889859250365.411,1030482257898.25,1016696893390.38,1009051505055.59
3,Africa Eastern and Southern,AFE,GDP,GDP per capita (current US$),162.913034498773,162.551683028333,172.002459991871,199.189238183763,179.387798934375,198.230367947484,...,1643.60556870139,1792.90750512756,1759.18239469392,1730.39468555029,1719.18372097247,1538.55226805656,1443.69237090812,1628.58678812124,1564.73433998697,1512.2705529912
4,Africa Western and Central,AFW,GDP,GDP (current US$),10404135069.15,11127894641.0191,11943187848.3043,12676330764.6917,13838369295.2313,14862225759.9135,...,597129288864.775,680455985950.414,736039861278.13,832216894624.603,892497905712.366,766957955078.105,690545418736.157,683748014299.694,766359667820.703,794719102944.191


In [825]:
all_gdp_df_formatted.dtypes

Country         object
Country Code    object
Type            object
Category        object
1960            object
                 ...  
2015            object
2016            object
2017            object
2018            object
2019            object
Length: 64, dtype: object

In [826]:
all_gdp_df_formatted['Category'].value_counts()

GDP per capita (current US$)    266
Population, total               266
GDP (current US$)               266
Name: Category, dtype: int64

In [827]:
# Paring down the countries contain to match the all_co_melted
pared_gdp_df = all_gdp_df_formatted[all_gdp_df_formatted['Country Code'].isin(all_co_melted['Country Code'])]

In [828]:
# Melt the data frame, id_vars is what columns to keep, value_vars is what columns to melt
gdp_melted = pd.melt(pared_gdp_df, id_vars=['Country', 'Country Code', 'Type', 'Category'],
                     var_name='Year', value_name='Value')

gdp_melted.head()

,Country,Country Code,Type,Category,Year,Value
0,Afghanistan,AFG,GDP,GDP (current US$),1960,537777811.111111
1,Afghanistan,AFG,GDP,GDP per capita (current US$),1960,62.3693745050559
2,Albania,ALB,GDP,GDP (current US$),1960,..
3,Albania,ALB,GDP,GDP per capita (current US$),1960,..
4,Algeria,DZA,GDP,GDP (current US$),1960,2723593384.78054


In [829]:
# Making sure these two total lines aren't present in this dataset
gdp_melted = gdp_melted[gdp_melted['Country'] != 'World']
gdp_melted = gdp_melted[gdp_melted['Country'] != 'International transport']

In [830]:
missing_values = gdp_melted.loc[gdp_melted['Value']=='..']
missing_values

,Country,Country Code,Type,Category,Year,Value
2,Albania,ALB,GDP,GDP (current US$),1960,..
3,Albania,ALB,GDP,GDP per capita (current US$),1960,..
6,Andorra,AND,GDP,GDP (current US$),1960,..
7,Andorra,AND,GDP,GDP per capita (current US$),1960,..
8,Angola,AGO,GDP,GDP (current US$),1960,..
...,...,...,...,...,...,...
36799,South Sudan,SSD,GDP,GDP per capita (current US$),2019,..
36860,"Venezuela, RB",VEN,GDP,GDP (current US$),2019,..
36861,"Venezuela, RB",VEN,GDP,GDP per capita (current US$),2019,..
36868,"Yemen, Rep.",YEM,GDP,GDP (current US$),2019,..


In [831]:
# What countries and 

In [832]:
gdp_melted.dtypes

Country         object
Country Code    object
Type            object
Category        object
Year            object
Value           object
dtype: object

In [833]:
# Converting year to Int and GDP_Value to float, replace ".." with NaN
gdp_melted[['Year']] = gdp_melted[['Year']].astype(int).copy()
gdp_melted['Value'] = pd.to_numeric(gdp_melted['Value'], errors='coerce').copy()


In [834]:
# Shortening the time frame from 1999-2019
gdp_melted_short = gdp_melted[gdp_melted['Year'] >= 1999]

In [835]:
# Reorder the columns
gdp_melted_almost_ready = gdp_melted_short[['Country', 'Country Code', 'Year', 'Type', 'Category', 'Value']]

In [836]:
# What country codes have null values?
missing_df = gdp_melted_almost_ready[gdp_melted_almost_ready['Value'].isnull()]
missing_df['Country Code'].value_counts()


PRK    42
VGB    42
SOM    28
SSD    26
SXM    26
CUW    24
NRU    22
ERI    16
VEN    10
AFG     6
STP     4
TCA     4
YEM     2
MNE     2
LBR     2
PLW     2
TLS     2
Name: Country Code, dtype: int64

In [837]:
# Pull the countries with null values from the GDP data
missing_countries = missing_df['Country'].unique()
gdp_melted_ready = gdp_melted_almost_ready[~gdp_melted_almost_ready['Country'].isin(missing_countries)]

### Make sure the CO2 df and GDP df have the same countries in them
All the countries from the orginal CO2 data should be represented with GDP data and there should be no extraneous countries with only GDP data in the set. 

In [838]:
# Get unique country codes for each df and compare
co_codes = all_co_ready['Country Code'].unique() 
gdp_codes = gdp_melted_ready['Country Code'].unique()

# Are there any countries in the CO2 df that are not in the per capita df?
co_not_in_gdp = []
for code in co_codes:
    if code not in gdp_codes:
        co_not_in_gdp.append(code)
print(f"There are {len(co_not_in_gdp)} countries in the CO2 df that are not in the GDP df.\
      The country not in the other data frame is: {co_not_in_gdp}")

# Show the countries that are not in the per capita df
co_not_in_gdp

# Are there any countries in the per capita df that are not in the CO2 df?
gdp_not_in_co = []   
for code in gdp_codes:
    if code not in co_codes:
        gdp_not_in_co.append(code)   
print(f"There are {len(gdp_not_in_co)} countries in the GDP df that are not in the CO2 df.\
      The country not in the other data frame is: {gdp_not_in_co}")

There are 37 countries in the CO2 df that are not in the GDP df.      The country not in the other data frame is: ['AFG', 'AIA', 'ATA', 'BES', 'VGB', 'CXR', 'COK', 'CUW', 'PRK', 'ERI', 'PCZ', 'GUF', 'GLP', 'KSV', 'LBR', 'MTQ', 'MYT', 'MNE', 'MSR', 'NRU', 'NIU', 'PLW', 'SSD', 'REU', 'SHN', 'SXM', 'STP', 'SOM', 'SPM', 'TWN', 'TLS', 'TCA', 'VEN', 'WLF', 'YEM', 'XIT', 'WLD']
There are 0 countries in the GDP df that are not in the CO2 df.      The country not in the other data frame is: []


In [839]:
# Drop the countries in the CO2 df that are not in the GDP df
all_co_ready = all_co_ready[~all_co_ready['Country Code'].isin(co_not_in_gdp)]


### Merge the CO2 and GDP dataframes

In [840]:
all_co_ready.head()

,Country,Country Code,Year,Type,Category,Value
521,Albania,ALB,1999,Total CO2,Total,2.986196
522,Albania,ALB,2000,Total CO2,Total,3.024926
523,Albania,ALB,2001,Total CO2,Total,3.220656
524,Albania,ALB,2002,Total CO2,Total,3.748272
525,Albania,ALB,2003,Total CO2,Total,4.303499


In [841]:
gdp_melted_ready.head()

,Country,Country Code,Year,Type,Category,Value
24104,Albania,ALB,1999,GDP,GDP (current US$),3.212122e+09
24105,Albania,ALB,1999,GDP,GDP per capita (current US$),1.033243e+03
24106,Algeria,DZA,1999,GDP,GDP (current US$),4.864065e+10
24107,Algeria,DZA,1999,GDP,GDP per capita (current US$),1.602864e+03
24108,Andorra,AND,1999,GDP,GDP (current US$),1.239876e+09


In [842]:
# Merge the CO2 and GDP data frames
co_gdp_df = pd.merge(all_co_ready, gdp_melted_ready, how='outer', on=['Country', 'Country Code', 'Year', 'Type', 'Category', 'Value'])

In [843]:
co_gdp_df.head()

,Country,Country Code,Year,Type,Category,Value
0,Albania,ALB,1999,Total CO2,Total,2.986196
1,Albania,ALB,2000,Total CO2,Total,3.024926
2,Albania,ALB,2001,Total CO2,Total,3.220656
3,Albania,ALB,2002,Total CO2,Total,3.748272
4,Albania,ALB,2003,Total CO2,Total,4.303499


In [844]:
# Find all NaN values and replace them with None, as NaN will error when it gets to the html
co_gdp_df = co_gdp_df.replace(np.nan, None)

In [889]:
# Test
test = co_gdp_df[co_gdp_df['Country Code'] == 'USA']

test['Category'].value_counts()

Total                           46
Oil                             46
Cement                          46
Gas                             46
Other                           46
Flaring                         46
Coal                            46
GDP per capita (current US$)    21
Population, total               21
GDP (current US$)               21
Name: Category, dtype: int64

### Adding Continent data onto the CO2/GDP/POP dataframe
One way we want to organize and view all this country data is by continent. What we are really looking for is just the continent code, the name wont be maintained in the database.

In [846]:
# Read in the continent data
cont_df = pd.read_csv('data/country-and-continent-codes-list-csv.csv')

cont_df.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [847]:
print("Missing values in 'Two_Letter_Country_Code' column:", cont_df['Two_Letter_Country_Code'].isna().sum())


Missing values in 'Two_Letter_Country_Code' column: 1


In [848]:
# Drop the rows with missing values
cont_df = cont_df.dropna(subset=['Two_Letter_Country_Code'])

In [849]:
# Rename the columns to match the CO2 data
cont_df = cont_df.rename(columns = {'Three_Letter_Country_Code':'Country Code', 'Continent_Name':'Continent', 'Continent_Code':'Continent Code'})

In [850]:
# Check for duplicate country codes in the continent data frame
cont_df['Country Code'].value_counts()
# Drop the duplicate country codes
cont_df = cont_df.drop_duplicates(subset=['Country Code'])

In [851]:
cont_df.head()

,Continent,Continent Code,Country_Name,Two_Letter_Country_Code,Country Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [852]:
print("Columns in co_gdp_df:", co_gdp_df.columns)
print("Columns in cont_df:", cont_df.columns)

Columns in co_gdp_df: Index(['Country', 'Country Code', 'Year', 'Type', 'Category', 'Value'], dtype='object')
Columns in cont_df: Index(['Continent', 'Continent Code', 'Country_Name',
       'Two_Letter_Country_Code', 'Country Code', 'Country_Number'],
      dtype='object')


In [853]:
# Bringing in continent data on to the co2/gdp/pop data frame, only bringing the Continent_Name column and the continent_Code column 
co_gdp_cont_df = pd.merge(co_gdp_df, cont_df[['Continent', 'Continent Code', 'Country Code']], how='left', left_on='Country Code', right_on='Country Code')


In [854]:
co_gdp_cont_df.head()

,Country,Country Code,Year,Type,Category,Value,Continent,Continent Code
0,Albania,ALB,1999,Total CO2,Total,2.986196,Europe,EU
1,Albania,ALB,2000,Total CO2,Total,3.024926,Europe,EU
2,Albania,ALB,2001,Total CO2,Total,3.220656,Europe,EU
3,Albania,ALB,2002,Total CO2,Total,3.748272,Europe,EU
4,Albania,ALB,2003,Total CO2,Total,4.303499,Europe,EU


In [855]:
# Find where continent code is null
co_gdp_cont_df[co_gdp_cont_df['Continent Code'].isnull()]

,Country,Country Code,Year,Type,Category,Value,Continent,Continent Code
92,Antigua and Barbuda,ATG,1999,Total CO2,Total,2.821280e-01,North America,NaN
93,Antigua and Barbuda,ATG,2000,Total CO2,Total,2.931200e-01,North America,NaN
94,Antigua and Barbuda,ATG,2001,Total CO2,Total,2.967840e-01,North America,NaN
95,Antigua and Barbuda,ATG,2002,Total CO2,Total,3.297600e-01,North America,NaN
96,Antigua and Barbuda,ATG,2003,Total CO2,Total,3.517440e-01,North America,NaN
...,...,...,...,...,...,...,...,...
72672,St. Kitts and Nevis,KNA,2019,GDP,"Population, total",4.771200e+04,North America,NaN
72673,St. Lucia,LCA,2019,GDP,"Population, total",1.785830e+05,North America,NaN
72674,St. Vincent and the Grenadines,VCT,2019,GDP,"Population, total",1.049240e+05,North America,NaN
72685,Trinidad and Tobago,TTO,2019,GDP,"Population, total",1.519955e+06,North America,NaN


In [856]:
# Fill the Continent Code column where the Continent is North America with NA
co_gdp_cont_df.loc[co_gdp_cont_df['Continent'] == 'North America', 'Continent Code'] = 'NA'
# Fill the Continent Code column where the Country is Nambia with AF
co_gdp_cont_df.loc[co_gdp_cont_df['Country'] == 'Namibia', 'Continent Code'] = 'AF'


In [857]:
# Find where continent code is null
co_gdp_cont_df[co_gdp_cont_df['Continent Code'].isnull()]

,Country,Country Code,Year,Type,Category,Value,Continent,Continent Code


In [858]:
# Remove the continent name as we are only maintaining the continent code
co_gdp_cont_df_cleaned = co_gdp_cont_df.drop(columns=['Continent'], inplace=True)

In [859]:
co_gdp_cont_df.isnull().sum()

Country           0
Country Code      0
Year              0
Type              0
Category          0
Value             0
Continent Code    0
dtype: int64

### Standardizing the country names
As we have pulled in a few different data sources, there is some discrepancy in the names so we are cleaning those up.

In [860]:
# Find instances where Country Code has more than one Country associated with it
dup_country_names = co_gdp_cont_df.groupby('Country Code')['Country'].nunique().sort_values(ascending=False)
dup_country_names = dup_country_names.loc[dup_country_names.values > 1]

In [861]:
# Getting the dupulicate country codes to a list to use in mapping the correct country name to the country code
dup_country_names.index.to_list()

['KNA',
 'CPV',
 'LCA',
 'SYR',
 'BHS',
 'SWZ',
 'MAC',
 'GMB',
 'RUS',
 'FSM',
 'FRO',
 'PSE',
 'CIV',
 'COD',
 'COG',
 'EGY',
 'CZE',
 'HKG',
 'SVK',
 'USA',
 'LAO',
 'VCT',
 'TUR',
 'IRN',
 'VNM',
 'KGZ',
 'KOR']

In [862]:
country_code_mapping = {
    'KNA': 'Saint Kitts and Nevis',
    'PRK': 'North Korea',
    'TUR': 'Turkey',
    'FRO': 'Faroe Islands',
    'HKG': 'Hong Kong',
    'SYR': 'Syria',
    'RUS': 'Russia',
    'BHS': 'Bahamas',
    'MAC': 'Macao',
    'PSE': 'Palestine',
    'CPV': 'Cabo Verde',
    'SVK': 'Slovakia',
    'COD': 'Democratic Republic of the Congo',
    'FSM': 'Micronesia',
    'CUW': 'Curaçao',
    'LCA': 'Saint Lucia',
    'IRN': 'Iran',
    'COG': 'Republic of the Congo',
    'CIV': "Côte d'Ivoire",
    'CZE': 'Czech Republic',
    'LAO': 'Laos',
    'USA': 'United States',
    'KOR': 'South Korea',
    'VCT': 'Saint Vincent and the Grenadines',
    'VEN': 'Venezuela',
    'VNM': 'Vietnam',
    'GMB': 'Gambia',
    'EGY': 'Egypt',
    'YEM': 'Yemen',
    'KGZ': 'Kyrgyzstan',
    'SWZ': 'Eswatini'
}

In [863]:
# Create a new column 'Standardized Country Name' using the mapping
co_gdp_cont_df['Standardized Country Name'] = co_gdp_cont_df['Country Code'].map(country_code_mapping)

# Update the 'Country' column only where the 'Standardized Country Name' is not null
co_gdp_cont_df['Country'] = co_gdp_cont_df.apply(lambda row: row['Standardized Country Name'] if pd.notnull(row['Standardized Country Name']) else row['Country'], axis=1)

# You can now drop the 'Standardized Country Name' column as it is not needed anymore
co_gdp_cont_df.drop('Standardized Country Name', axis=1, inplace=True)

In [864]:
# Check to see if there are still any Country Codes with more than one Country associated with it
dup_country_names2 = co_gdp_cont_df.groupby('Country Code')['Country'].nunique().sort_values(ascending=False)
dup_country_names2 = dup_country_names2.loc[dup_country_names2.values > 1]
dup_country_names2

Series([], Name: Country, dtype: int64)

In [865]:
co_gdp_cont_df['Country Code'].nunique()

188

In [866]:
co_gdp_cont_df_cleaned = co_gdp_cont_df.copy()

In [867]:
# One last bit of formatting to make the data easier to read

# Make all values in the 'Value' column rounded to 6 decimal places
co_gdp_cont_df_cleaned = co_gdp_cont_df_cleaned.round({'Value': 6})

In [868]:
# check for null, na, np.nan values
co_gdp_cont_df_cleaned.isnull().sum()


Country           0
Country Code      0
Year              0
Type              0
Category          0
Value             0
Continent Code    0
dtype: int64

In [869]:
# Checking for different flavors of null or missing values
# The first .sum is for the number of columns with null values and the second .sum is for the total number of null values
null_values = co_gdp_cont_df_cleaned.isnull().sum().sum()
print(f'Total number of null values: {null_values}')

# Check for NA values
na_values = co_gdp_cont_df_cleaned.isna().sum().sum()
print(f'Total number of NA values: {na_values}')

# Check for np.NaN values
nan_values = co_gdp_cont_df_cleaned.isin([np.nan]).sum().sum()
print(f'Total number of np.NaN values: {nan_values}')

# Check for missing values
missing_values = null_values + na_values + nan_values
print(f'Total number of missing values: {missing_values}')

Total number of null values: 0
Total number of NA values: 0
Total number of np.NaN values: 0
Total number of missing values: 0


### Import to MongoDB
There is a single data frame that will be loaded into mongoDB as a flat structure.

In [870]:
# Convert the dataframes to a list of dictionaries
co_gdp_cont_df_to_load = co_gdp_cont_df_cleaned.to_dict('records')

In [871]:
# connect to the MongoDB client and create the database and collection

client = MongoClient('mongodb://localhost:27017/')
db = client['global_emissions_db']
collection = db['CO2_gdp_population']

# Insert the data into the collection
collection.insert_many(co_gdp_cont_df_to_load)


### Test the Database   

In [881]:
# Create a decorator to manage the connection to the MongoDB server
def with_mongo_client(func):
    def wrapper(*args, **kwargs):
        client = MongoClient("mongodb://localhost:27017/")
        try:
            return func(client, *args, **kwargs)
        finally:
            client.close()

    return wrapper

# Test fuctions for the databse 

# Show 3 random documents
@with_mongo_client
def show_three_random_documents(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor =  collection.aggregate([{'$sample': {'size': 3}}])
    return list(cursor)

# Get the total GDP for US in 2015
@with_mongo_client
def get_gdp_us_2015(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.find({'Country': 'United States', 'Year': 2015, 'Type': 'GDP', 'Category':'GDP (current US$)'}, {'_id': 0, 'Value': 1})
    return list(cursor)

# Get all the documents for 2009-2019 where the Type is GDP and the Category is GDP (current US$) with the value NaN
@with_mongo_client
def get_gdp_nan_1999_2019(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.find({'Year': {'$gte': 1999, '$lte': 2019}, 'Type': 'GDP', 'Category':'GDP (current US$)', 'Value': np.NaN}, {'_id': 0})
    return list(cursor)

# Get all the documents for 2009-2019 where the Type is Total CO2 and the Category is Total with the value NaN
@with_mongo_client
def get_co2_nan_1999_2019(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.find({'Year': {'$gte': 1999, '$lte': 2019}, 'Type': 'Total CO2', 'Category':'Total', 'Value': None}, {'_id': 0})
    return list(cursor)

# Get 3 random documents where the Year is 2019, the Type is GDP and the Category is Population, total
@with_mongo_client
def get_three_population_2019(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.aggregate([
        {'$match': {'Year': 2019, 'Type': 'GDP', 'Category': 'Population, total'}},
        {'$sample': {'size': 3}}
    ])
    return list(cursor)

# Get 3 random documents where the the Type is GDP and the Category is Population, total
@with_mongo_client
def get_three_pc_gdp(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.aggregate([
        {'$match': {'Type': 'GDP', 'Category': 'GDP per capita (current US$)'}},
        {'$sample': {'size': 3}}
    ])
    return list(cursor)

# Get all documents for a country code for a certain year, function takes those arguments
@with_mongo_client
def get_country_code_year(client, country_code, year):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.find({'Country Code': country_code, 'Year': year}, {'_id': 0})
    return list(cursor)

# Get # of unique countries in the database
@with_mongo_client
def get_unique_countries(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    cursor = collection.distinct('Country')
    return len(cursor)

In [882]:
get_unique_countries()

188

In [883]:
get_country_code_year('MAF', 2013)

[]

In [884]:
get_three_pc_gdp()

[{'_id': ObjectId('6440a080d0785497af3cbac5'),
  'Country': 'Liechtenstein',
  'Country Code': 'LIE',
  'Year': 2007,
  'Type': 'GDP',
  'Category': 'GDP per capita (current US$)',
  'Value': 130904.681844,
  'Continent Code': 'EU'},
 {'_id': ObjectId('6440a080d0785497af3cbc61'),
  'Country': 'Brunei Darussalam',
  'Country Code': 'BRN',
  'Year': 2008,
  'Type': 'GDP',
  'Category': 'GDP per capita (current US$)',
  'Value': 37426.668544,
  'Continent Code': 'AS'},
 {'_id': ObjectId('6440a080d0785497af3cd05f'),
  'Country': 'Czech Republic',
  'Country Code': 'CZE',
  'Year': 2017,
  'Type': 'GDP',
  'Category': 'GDP per capita (current US$)',
  'Value': 20636.199952,
  'Continent Code': 'EU'}]

In [885]:
show_three_random_documents()

[{'_id': ObjectId('6440a07fd0785497af3c3c1f'),
  'Country': 'Nepal',
  'Country Code': 'NPL',
  'Year': 2007,
  'Type': 'PerCapita CO2',
  'Category': 'Gas',
  'Value': 0.0,
  'Continent Code': 'AS'},
 {'_id': ObjectId('6440a07fd0785497af3bc884'),
  'Country': 'Slovakia',
  'Country Code': 'SVK',
  'Year': 2013,
  'Type': 'Total CO2',
  'Category': 'Total',
  'Value': 35.565563,
  'Continent Code': 'EU'},
 {'_id': ObjectId('6440a07fd0785497af3be0d5'),
  'Country': 'Dominica',
  'Country Code': 'DMA',
  'Year': 2005,
  'Type': 'Total CO2',
  'Category': 'Coal',
  'Value': 0.0,
  'Continent Code': 'NA'}]

In [877]:
get_three_population_2019()

[{'_id': ObjectId('6440a080d0785497af3cd62f'),
  'Country': 'Hong Kong',
  'Country Code': 'HKG',
  'Year': 2019,
  'Type': 'GDP',
  'Category': 'Population, total',
  'Value': 7507900.0,
  'Continent Code': 'AS'},
 {'_id': ObjectId('6440a080d0785497af3cd654'),
  'Country': 'Mexico',
  'Country Code': 'MEX',
  'Year': 2019,
  'Type': 'GDP',
  'Category': 'Population, total',
  'Value': 125085311.0,
  'Continent Code': 'NA'},
 {'_id': ObjectId('6440a080d0785497af3cd621'),
  'Country': 'Gabon',
  'Country Code': 'GAB',
  'Year': 2019,
  'Type': 'GDP',
  'Category': 'Population, total',
  'Value': 2242785.0,
  'Continent Code': 'AF'}]

In [886]:
client = MongoClient('mongodb://localhost:27017/')
db = client['global_emissions_db']

# How many documents are in the collection?
print(f' There are {db.CO2_gdp_population.count_documents({})} documents in the collection')

# How many 'Values' are NA, NaN or Null or None?
print(f" There are {db.CO2_gdp_population.count_documents({'Value': {'$in': [np.NaN, None, '', 'NA', 'N/A', '-','..', 'unknown']}})} documents with missing values")


 There are 72702 documents in the collection
 There are 0 documents with missing values


In [888]:
client = MongoClient('mongodb://localhost:27017/')
db = client['global_emissions_db']

# Get list of all the coutnry codes in the database
country_codes = db.CO2_gdp_population.distinct('Country Code')

print(country_codes)

['ABW', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAC', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD', 'MLI', 'MLT', 'MMR', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PNG', 'POL', 'PRI', 'PRT'

### Code for Deleting the DB if necessary

In [880]:
# Code for deleting all the collections in the database

# client = MongoClient('mongodb://localhost:27017/')
# db = client['global_emissions_db']

# # Delete all the collections
# for collection_name in db.list_collection_names():
#     db[collection_name].drop() 